This notebook is used to save calibration factors as correctionlib-compatible Json file


In [7]:
from correctionlib import schemav2 as schema

import correctionlib
from correctionlib import _core
import awkward as ak

In [31]:
"""
Define 2018 UL calibration from BeamSpot Constraint as defined in AN-2023/017
"""
year = 2018
# Define a correction with three input parameters
correction = schema.Correction(
    name="BS_ebe_mass_res_calibration",
    description="Dimuon Mass resolution calibration with BeamSpot Constraint correction applied",
    version=1,
    inputs=[
        schema.Variable(name="leading_mu_pt", type="real", description="Transverse momentum of the leading muon (GeV)"),
        schema.Variable(name="leading_mu_abseta", type="real", description="Absolute pseudorapidity of the leading muon"),
        schema.Variable(name="subleading_mu_abseta", type="real", description="Absolute pseudorapidity of the subleading muon")
    ],
    output=schema.Variable(name="correction_factor", type="real"),
    data=schema.MultiBinning(
        nodetype="multibinning",
        inputs=["leading_mu_pt", "leading_mu_abseta", "subleading_mu_abseta"],
        edges=[
            [30, 45, 52, 62, 200],  # Edges for leading_mu_pt
            [0, 0.9, 1.8, 2.4],  # Edges for leading_mu_abseta
            [0, 0.9, 1.8, 2.4]  # Edges for subleading_mu_abseta
        ],
        content=[ # Content array for correction factors
            1.220, 1.273, 1.181, # BB, BO, BE, 30–45
            1.220, 1.273, 1.181, # OB, OO, OE, 30–45 
            1.220, 1.273, 1.181, # EB, EO, EE  30–45 
            1.216, 1.175, 1.158, 1.231, 1.160, 1.094, 1.095, 1.061, 1.023, 1.246, 1.207, 1.099, 1.234, 1.203, 1.151, 1.170, 1.119, 1.108, 1.214, 1.197, 1.159, 1.152, 1.233, 1.179, 1.182, 1.089, 1.068, 
            # values from from table 16. From category 3 onwards, you just need to copy paste. The order on the table matches with correctionlib's order
        ],
        flow="clamp"  # Handles out-of-bounds input values
    )
)

# Create the correction set and add the correction
correction_set = schema.CorrectionSet(
    schema_version=schema.VERSION,
    corrections=[correction]
)

# Save the correction set to a JSON file
# with open("muon_correction.json", "w") as f:
#     f.write(correction_set.json(exclude_unset=True, indent=2))

json_name = f"res_calib_BS_correction_{year}UL.json"

with open(json_name, "w") as fout:
    fout.write(correction_set.json(exclude_unset=True))

# import gzip


# with gzip.open("mycorrections.json.gz", "wt") as fout:
#     fout.write(correction_set.json(exclude_unset=True))
# print("Muon correction schema created and saved as 'muon_correction.json'")


In [32]:
"""
validate the correctionlib using test cases
"""


# Load the correction set
correction_set = _core.CorrectionSet.from_file(json_name)

# Access the specific correction by name
correction = correction_set["BS_ebe_mass_res_calibration"]

# Create an evaluator for the correction
def evaluate_correction(leading_mu_pt, leading_mu_eta, subleading_mu_eta):
    return correction.evaluate(leading_mu_pt, leading_mu_eta, subleading_mu_eta)

# Test the evaluator with some example inputs
test_data = [
    [35.0, 0.5, 0.5],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 BB ->cat 0
    [35.0, 0.95, 0.5],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 OB ->cat 0
    [35.0, 2.0, 0.5],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 EB ->cat 0
    [35.0, 0.5, 0.95],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 BO -> cat1
    [35.0, 1.0, 0.95],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 OO -> cat1
    [36.0, 2.3, 0.95],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 EO -> cat1
    [35.0, 0.5, 2.3],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 BE -> cat2
    [35.0, 1.0, 2.3],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 OE -> cat2
    [36.0, 2.3, 2.5],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 EE -> cat2
    [60.0, 0.5, 0.5],     # leading_mu_pt = 47, leading_mu_eta = 1.0, subleading_mu_eta = 1.5 BB -> cat12
    [46.0, 0.5, 0.5],     # leading_mu_pt = 47, leading_mu_eta = 1.0, subleading_mu_eta = 1.5 BB -> cat3
    [66.0, 0.5, 0.95],     # leading_mu_pt = 47, leading_mu_eta = 1.0, subleading_mu_eta = 1.5 BO -> cat22
    [66.0, 0.5, 2.2],     # leading_mu_pt = 47, leading_mu_eta = 1.0, subleading_mu_eta = 1.5 BO -> cat23
]
test_data = ak.Array(test_data)
# Evaluate the correction for each input triplet
results = [evaluate_correction(pt, eta1, eta2) for pt, eta1, eta2 in test_data]

# Print results
for (pt, eta1, eta2), result in zip(test_data, results):
    print(f"leading_mu_pt: {pt}, leading_mu_eta: {eta1}, subleading_mu_eta: {eta2}, scale factor: {result}")

leading_mu_pt: 35.0, leading_mu_eta: 0.5, subleading_mu_eta: 0.5, scale factor: 1.22
leading_mu_pt: 35.0, leading_mu_eta: 0.95, subleading_mu_eta: 0.5, scale factor: 1.22
leading_mu_pt: 35.0, leading_mu_eta: 2.0, subleading_mu_eta: 0.5, scale factor: 1.22
leading_mu_pt: 35.0, leading_mu_eta: 0.5, subleading_mu_eta: 0.95, scale factor: 1.273
leading_mu_pt: 35.0, leading_mu_eta: 1.0, subleading_mu_eta: 0.95, scale factor: 1.273
leading_mu_pt: 36.0, leading_mu_eta: 2.3, subleading_mu_eta: 0.95, scale factor: 1.273
leading_mu_pt: 35.0, leading_mu_eta: 0.5, subleading_mu_eta: 2.3, scale factor: 1.181
leading_mu_pt: 35.0, leading_mu_eta: 1.0, subleading_mu_eta: 2.3, scale factor: 1.181
leading_mu_pt: 36.0, leading_mu_eta: 2.3, subleading_mu_eta: 2.5, scale factor: 1.181
leading_mu_pt: 60.0, leading_mu_eta: 0.5, subleading_mu_eta: 0.5, scale factor: 1.246
leading_mu_pt: 46.0, leading_mu_eta: 0.5, subleading_mu_eta: 0.5, scale factor: 1.216
leading_mu_pt: 66.0, leading_mu_eta: 0.5, subleading_

In [34]:
"""
Define 2017 UL calibration from BeamSpot Constraint as defined in AN-2023/017
"""
year = 2017
# Define a correction with three input parameters
correction = schema.Correction(
    name="BS_ebe_mass_res_calibration",
    description="Dimuon Mass resolution calibration with BeamSpot Constraint correction applied",
    version=1,
    inputs=[
        schema.Variable(name="leading_mu_pt", type="real", description="Transverse momentum of the leading muon (GeV)"),
        schema.Variable(name="leading_mu_abseta", type="real", description="Absolute pseudorapidity of the leading muon"),
        schema.Variable(name="subleading_mu_abseta", type="real", description="Absolute pseudorapidity of the subleading muon")
    ],
    output=schema.Variable(name="correction_factor", type="real"),
    data=schema.MultiBinning(
        nodetype="multibinning",
        inputs=["leading_mu_pt", "leading_mu_abseta", "subleading_mu_abseta"],
        edges=[
            [30, 45, 52, 62, 200],  # Edges for leading_mu_pt
            [0, 0.9, 1.8, 2.4],  # Edges for leading_mu_abseta
            [0, 0.9, 1.8, 2.4]  # Edges for subleading_mu_abseta
        ],
        content=[ # Content array for correction factors
            1.164, 1.052, 1.097, # BB, BO, BE, 30–45
            1.164, 1.052, 1.097, # OB, OO, OE, 30–45 
            1.164, 1.052, 1.097, # EB, EO, EE  30–45 
            1.125, 1.123, 1.060, 1.131, 1.072, 1.038, 0.991, 1.045, 0.970, 1.149, 1.181, 1.159, 1.146, 1.191, 1.117, 1.139, 1.138, 1.126, 1.129, 1.144, 1.070, 1.178, 1.198, 1.057, 1.036, 1.066, 1.019, 
            # values from from table 16. From category 3 onwards, you just need to copy paste. The order on the table matches with correctionlib's order
        ],
        flow="clamp"  # Handles out-of-bounds input values
    )
)

# Create the correction set and add the correction
correction_set = schema.CorrectionSet(
    schema_version=schema.VERSION,
    corrections=[correction]
)

# Save the correction set to a JSON file
# with open("muon_correction.json", "w") as f:
#     f.write(correction_set.json(exclude_unset=True, indent=2))

json_name = f"res_calib_BS_correction_{year}UL.json"

with open(json_name, "w") as fout:
    fout.write(correction_set.json(exclude_unset=True))

# import gzip


# with gzip.open("mycorrections.json.gz", "wt") as fout:
#     fout.write(correction_set.json(exclude_unset=True))
# print("Muon correction schema created and saved as 'muon_correction.json'")


In [36]:
"""
validate the correctionlib using test cases
"""


# Load the correction set
correction_set = _core.CorrectionSet.from_file(json_name)

# Access the specific correction by name
correction = correction_set["BS_ebe_mass_res_calibration"]

# Create an evaluator for the correction
def evaluate_correction(leading_mu_pt, leading_mu_eta, subleading_mu_eta):
    return correction.evaluate(leading_mu_pt, leading_mu_eta, subleading_mu_eta)

# Test the evaluator with some example inputs
test_data = [
    [35.0, 0.5, 0.5],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 BB ->cat 0
    [35.0, 0.95, 0.5],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 OB ->cat 0
    [35.0, 2.0, 0.5],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 EB ->cat 0
    [35.0, 0.5, 0.95],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 BO -> cat1
    [35.0, 1.0, 0.95],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 OO -> cat1
    [36.0, 2.3, 0.95],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 EO -> cat1
    [35.0, 0.5, 2.3],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 BE -> cat2
    [35.0, 1.0, 2.3],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 OE -> cat2
    [36.0, 2.3, 2.5],     # leading_mu_pt = 35, leading_mu_eta = 0.5, subleading_mu_eta = 0.5 EE -> cat2
    [60.0, 0.5, 0.5],     # leading_mu_pt = 47, leading_mu_eta = 1.0, subleading_mu_eta = 1.5 BB -> cat12
    [46.0, 0.5, 0.5],     # leading_mu_pt = 47, leading_mu_eta = 1.0, subleading_mu_eta = 1.5 BB -> cat3
    [66.0, 0.5, 0.95],     # leading_mu_pt = 47, leading_mu_eta = 1.0, subleading_mu_eta = 1.5 BO -> cat22
    [66.0, 0.5, 2.2],     # leading_mu_pt = 47, leading_mu_eta = 1.0, subleading_mu_eta = 1.5 BO -> cat23
]
test_data = ak.Array(test_data)
# Evaluate the correction for each input triplet
results = [evaluate_correction(pt, eta1, eta2) for pt, eta1, eta2 in test_data]

# Print results
for (pt, eta1, eta2), result in zip(test_data, results):
    print(f"leading_mu_pt: {pt}, leading_mu_eta: {eta1}, subleading_mu_eta: {eta2}, scale factor: {result}")

leading_mu_pt: 35.0, leading_mu_eta: 0.5, subleading_mu_eta: 0.5, scale factor: 1.164
leading_mu_pt: 35.0, leading_mu_eta: 0.95, subleading_mu_eta: 0.5, scale factor: 1.164
leading_mu_pt: 35.0, leading_mu_eta: 2.0, subleading_mu_eta: 0.5, scale factor: 1.164
leading_mu_pt: 35.0, leading_mu_eta: 0.5, subleading_mu_eta: 0.95, scale factor: 1.052
leading_mu_pt: 35.0, leading_mu_eta: 1.0, subleading_mu_eta: 0.95, scale factor: 1.052
leading_mu_pt: 36.0, leading_mu_eta: 2.3, subleading_mu_eta: 0.95, scale factor: 1.052
leading_mu_pt: 35.0, leading_mu_eta: 0.5, subleading_mu_eta: 2.3, scale factor: 1.097
leading_mu_pt: 35.0, leading_mu_eta: 1.0, subleading_mu_eta: 2.3, scale factor: 1.097
leading_mu_pt: 36.0, leading_mu_eta: 2.3, subleading_mu_eta: 2.5, scale factor: 1.097
leading_mu_pt: 60.0, leading_mu_eta: 0.5, subleading_mu_eta: 0.5, scale factor: 1.149
leading_mu_pt: 46.0, leading_mu_eta: 0.5, subleading_mu_eta: 0.5, scale factor: 1.125
leading_mu_pt: 66.0, leading_mu_eta: 0.5, subleadi